# Entanglement Feature Learning

## EFL

### Convension of Coupling Constants
The original EFL model (for the Ising case) is
$$E[\sigma]=-\sum_{\langle ij\rangle}J_{ij} \chi(\sigma_i^{-1}\sigma_j),$$
where $J_{ij}=\ln D_{ij}$ and $\sigma_{i}$ takes values in the $S_2$ group, and the cycle trace $\chi$ maps $()\to2$, $(1,2)\to1$. The energy difference is one unit of $J_{ij}$. Note that each bond $\langle ij\rangle$ is added only once in the summation. Now for easier treatment in numerics, we map the $S_2$ spin $\sigma_i$ to a $\mathbb{Z}_2$ spin $s_i$, s.t. the energy model becomes
$$E[s]=-\frac{1}{2}\sum_{\langle ij\rangle}J_{ij}s_is_j=-\frac{1}{4}\sum_{i j}J_{ij}s_is_j.$$
We then define
$$K_{ij}=\frac{1}{2}J_{ij}=\frac{1}{2}\ln D_{ij},$$
and rewrite the energy model as
$$E[s]=-\frac{1}{2}\sum_{ij}K_{ij}s_is_j,$$
where the summation double counts a bond, and $s_{i}=\pm1$.

### Deep Boltzmann Machine (Model)

Layers: $s^{0},s^{1},\cdots$, where $s^0$ is the visible layer, rests are hidden layers. The energy model:
$$E[s]=-\sum_{l=1:L}\sum_{i j}s_i^{l-1}K_{ij}^l s_j^l.$$

## Restricted Boltzman Machine (RBM)

### RBM Kernel
Differences to standard RBM:
- the binary units takes values $\pm1$ instead of $0,1$.
- **unbiased**, RTN of pure state are not biased, so the only variational parameter is the weight matrix.
- elements in the weight matrixes are **positive** definite, because they correspnd to the logarithmic bond dimension.

Initialization of weight matrix. Wish to:
- reflect the locality.
- ferromagnetic, tune to critical.
- with some randomness.

Gibbs sampling:
- propagating up:
$$h_j\leftarrow\tanh \left\{\begin{array}{cc}\sum_{i}W_{ij}v_i & \text{default, top},\\ 2\sum_{i}W_{ij}v_i & \text{bottom, intermediate},\end{array}\right.$$
- propagating down:
$$v_i\leftarrow\tanh \left\{\begin{array}{cc}\sum_{j}W_{ij}h_j & \text{default, bottom},\\ 2\sum_{j}W_{ij}h_j & \text{top, intermediate},\end{array}\right.$$

Update rule ($\lambda_\text{l}$ learning rate, $\lambda_\text{f}$ forgetting rate):
$$W\leftarrow \text{relu}[(1-\lambda_\text{f})W+\lambda_\text{l} (v^{\intercal}(0) h(0)-v^{\intercal}(\infty) h(\infty))]$$
- If any element in $W$ becomes negative, the element is set to zero.
- Forgetting rate can be used to control the bond dimension.

In [288]:
%run 'EFL.py'
rbm = RBM(4,2)

In [287]:
print(rbm.learn(numpy.asarray(numpy.random.randint(0,2,(40,4))*2-1,dtype=float),0.1,0.01))
rbm.W.get_value()

0.7453134201113543


array([[ 0.165,  0.011],
       [ 0.708,  0.176],
       [ 0.120,  0.285],
       [ 0.071,  0.809]])

In [366]:
print(rbm.learn(numpy.array([[1,1,1,1],[-1,1,-1,1]]*10,dtype=float),0.1,0.01))
rbm.W.get_value()

1.081018265501512


array([[ 1.194,  0.031],
       [ 0.005,  0.940],
       [ 1.189,  0.003],
       [ 0.000,  0.946]])

#### Test by Ideal States
* trivial product state: deep PM, all Ising configuration equal weight.
* random state (maximally thermalized): deep FM, all up or all down.

In [16]:
%run EFL.py
train_FM = numpy.array([[1,1,1,1],[-1,-1,-1,-1]]*200,dtype=float)
train_AFM = numpy.array([[1,-1,1,-1],[-1,1,-1,1]]*200,dtype=float)
train_PM = numpy.asarray(numpy.random.randint(0,2,(400,4))*2-1,dtype=float)
#lr_table = [0.2,0.3,0.25,0.15,0.1]
#fr_table = [0.05,0.01,0.,0.,0.]
lr_table = [0.5,0.3,0.2,0.15,0.1,0.1,0.1]
fr_table = [0.,0.,0.,0.,0.,0.,0.]

FM, maximally thermal.

In [20]:
rbm = RBM(Nv=4,Nh=2)
for epoch in range(7):
    cost = rbm.learn(train_FM,lr_table[epoch],fr_table[epoch])
    print('Epoch %d: '%epoch, 'cost = %f'%cost)
rbm.W.get_value()

Epoch 0:  cost = 0.946498
Epoch 1:  cost = 0.901662
Epoch 2:  cost = 0.687882
Epoch 3:  cost = 0.599588
Epoch 4:  cost = 0.612765
Epoch 5:  cost = 0.556824
Epoch 6:  cost = 0.536112


array([[ 0.845,  0.575],
       [ 1.516,  0.517],
       [ 0.737,  0.613],
       [ 0.642,  0.933]])

PM, trivial product state.

In [31]:
rbm = RBM(Nv=4,Nh=2)
for epoch in range(7):
    cost = rbm.learn(train_PM,lr_table[epoch],fr_table[epoch])
    print('Epoch %d: '%epoch, 'cost = %f'%cost)
rbm.W.get_value()

Epoch 0:  cost = 1.026209
Epoch 1:  cost = 1.021890
Epoch 2:  cost = 1.005196
Epoch 3:  cost = 0.953806
Epoch 4:  cost = 0.973413
Epoch 5:  cost = 0.966338
Epoch 6:  cost = 0.966973


array([[ 1.397,  0.003],
       [ 0.089,  0.006],
       [ 0.000,  0.400],
       [ 0.018,  0.692]])

### Convolutional RBM
Assumption: translational symmetry (or in statistical sense under disorder average). Weights are shared among kernels for each layer and each group. This makes the algorithm scalable. For disordered system, this will learn the disorder averaged entanglement features.

Consider a 1D free fermion CFT, the Renyi entropy given by (acorrding to Calabrese and Cardy 2004, 2009)
$$S\propto\sum_{i,j}\ln|u_i-v_j|-\sum_{i<j}\ln|u_i-u_j|-\sum_{i<j}\ln|v_i-v_j|.$$
$u_i$ and $v_i$ are positions of kinks and antikinks in the Ising configuraiton.

In [26]:
from itertools import combinations
def entropy_CFT(kinks):
    us = kinks[0::2]
    vs = kinks[1::2]
    Suv = sum(numpy.log(abs(u-v)) for u in us for v in vs)
    Suu = sum(numpy.log(abs(u1-u2)) for u1, u2 in combinations(us,2))
    Svv = sum(numpy.log(abs(v1-v2)) for v1, v2 in combinations(vs,2))
    S = Suv-Suu-Svv
    return S
entropy_CFT([3,10])

1.9459101490553132

## Deep Boltzmann Machine

Create a DBM and prepare training set.

In [103]:
%run 'EFL.py'
dbm = DBM([8,4,2,1])
test_samples = [
    [+1,+1,+1,+1,+1,+1,+1,+1],
    [-1,-1,-1,-1,-1,-1,-1,-1],
    [+1,+1,+1,+1,-1,-1,-1,-1],
    [-1,-1,-1,-1,+1,+1,+1,+1],
    #[+1,+1,-1,-1,-1,-1,+1,+1],
    #[-1,-1,+1,+1,+1,+1,-1,-1]
]
data = Server(test_samples*100,15)
#data = Server(numpy.random.randint(0,2,(200,8))*2-1,10)

Pretrain and show weights.

In [104]:
dbm.pretrain(data,lrs=[0.5,0.3,0.2,0.15],frs=[0.05,0.01])

Pretraining layer 0:
    Epoch 0: cost = 0.529616
    Epoch 1: cost = 0.229190
    Epoch 2: cost = 0.082441
    Epoch 3: cost = 0.031849
    Epoch 4: cost = 0.030100
    Epoch 5: cost = 0.009616
    Epoch 6: cost = 0.008593
Pretraining layer 1:
    Epoch 0: cost = 1.564733
    Epoch 1: cost = 1.475176
    Epoch 2: cost = 1.211319
    Epoch 3: cost = 0.900614
    Epoch 4: cost = 0.786393
    Epoch 5: cost = 0.653053
    Epoch 6: cost = 0.716692
Pretraining layer 2:
    Epoch 0: cost = 1.303247
    Epoch 1: cost = 1.039667
    Epoch 2: cost = 1.036992
    Epoch 3: cost = 1.023406
    Epoch 4: cost = 1.023770


In [105]:
for rbm in dbm.rbms:
    print(rbm.W.get_value())

[[ 1.312  1.206  0.067  0.067]
 [ 1.413  1.294  0.013  0.013]
 [ 1.311  1.427  0.081  0.082]
 [ 1.128  1.282  0.054  0.056]
 [ 0.040  0.040  1.364  1.280]
 [ 0.000  0.000  1.271  1.280]
 [ 0.040  0.040  1.280  1.373]
 [ 0.000  0.000  1.287  1.349]]
[[ 1.258  0.027]
 [ 1.258  0.027]
 [ 0.027  1.318]
 [ 0.027  1.320]]
[[ 0.017]
 [ 0.135]]


In [35]:
f = theano.function([dbm.input],dbm.rbms[1].output)
f([[+1,+1,+1,+1,+1,+1,+1,+1],
   [-1,-1,-1,-1,-1,-1,-1,-1],
   [-1,-1,-1,-1,+1,+1,+1,+1],
   [+1,+1,+1,+1,-1,-1,-1,-1]])

array([[ 1.000,  1.000],
       [-1.000, -1.000],
       [-1.000,  1.000],
       [ 1.000, -1.000]])

Fine-tuning and show weights.

In [36]:
dbm.finetune(data,10,lrs=[0.5,0.5,0.4,0.3,0.2,0.1])

    Epoch 0: cost = 0.005486
    Epoch 1: cost = 0.003521
    Epoch 2: cost = 0.003186
    Epoch 3: cost = 0.001988
    Epoch 4: cost = 0.001897
    Epoch 5: cost = 0.001822
    Epoch 6: cost = 0.001761
    Epoch 7: cost = 0.001747
    Epoch 8: cost = 0.001740
    Epoch 9: cost = 0.001734


In [37]:
for rbm in dbm.rbms:
    print(rbm.W.get_value())

[[ 1.712  1.722  0.000  0.000]
 [ 1.647  1.652  0.000  0.000]
 [ 1.766  1.759  0.000  0.000]
 [ 1.528  1.592  0.000  0.000]
 [ 0.000  0.000  1.680  1.656]
 [ 0.000  0.000  1.755  1.691]
 [ 0.000  0.000  1.731  1.727]
 [ 0.000  0.000  1.657  1.743]]
[[ 1.293  0.000]
 [ 1.288  0.000]
 [ 0.000  1.330]
 [ 0.000  1.318]]
[[ 0.018]
 [ 0.000]]


## Random Heisenberg Model

Random Heisenberg model:
$$H=J_1\sum_{\langle ij\rangle}\vec{S}_i\cdot\vec{S}_j+J_2\sum_{\langle\langle ij\rangle\rangle}(S_i^+S_j^-+h.c.)+\sum_i h_i S_i^z,$$
where $h_i\in[-W,W]$ is independently random on each site. According to Vedica, the MBL-ETH transition happens around $W_c\simeq 5$. We assume periodic boundary condition, and ED the Hamiltonian in the total-$S^z$ zero sector.

`Entanglement` object takes the many-body wave function given by `random_Heisenberg`. It than perform the MPS decomposition, and construct  entanglement entropy (2nd Renyi, in unit of bit) for all entanglement regions (secified by Ising configruation). The result is stored as a dict in `Entanglement.Sdict`.

In [64]:
%run 'EFL.py'
ent = Entanglement(random_Heisenberg(L = 8, W = 1.))
d = {k:[] for k in range(0,ent.L//2+1)}
for config, W in zip(ent.configs, ent.weights):
    d[min(tuple(config).count(1),tuple(config).count(-1))].append(-numpy.log2(W))
{k:(numpy.mean(Ss), numpy.std(Ss)/numpy.mean(Ss)) for k, Ss in d.items()}

{0: (6.4068530076298373e-16, 0.0),
 1: (0.96028850843789637, 0.039106523833883598),
 2: (1.8123988751016264, 0.05916211303357078),
 3: (2.4642948420407103, 0.059053162762554776),
 4: (2.725657982673686, 0.058492579937884204)}

Generate random configurations for training set.

In [70]:
subset = numpy.random.choice(len(ent.configs),len(ent.configs)//10,replace=False) 
ent.rand_configs(8,subset=subset, beta=5.)

array([[ 1, -1, -1, -1, -1,  1, -1, -1],
       [ 1, -1, -1, -1, -1,  1, -1, -1],
       [-1,  1,  1, -1,  1, -1, -1,  1],
       [-1,  1,  1, -1,  1,  1, -1,  1],
       [ 1,  1,  1, -1,  1, -1, -1, -1],
       [ 1, -1, -1, -1, -1,  1, -1, -1],
       [ 1,  1, -1,  1,  1,  1,  1, -1],
       [-1, -1, -1, -1,  1, -1,  1, -1]])

In [71]:
prob = ent.weights[subset]
prob = (prob/prob.max())**5.
prob

array([ 0.118,  0.443,  0.040,  0.048,  0.012,  0.014,  0.368,  0.083,
        0.047,  0.046,  0.057,  0.010,  0.438,  1.000,  0.056,  0.013,
        0.074,  0.103,  0.030,  0.061,  0.061,  0.072,  0.026,  0.031,
        0.050])

In [95]:
Ls = [8,4,2,1]
ent = Entanglement(random_Heisenberg(L = Ls[0], W=1.))
train = numpy.random.choice(len(ent.configs),len(ent.configs)//2,replace=False) 
test = numpy.array(list(set(range(len(ent.configs)))-set(train)))
data = Server(ent.rand_configs(2000,subset=train,beta=1.),batch_size=20)
dbm = DBM(Ls)
dbm.pretrain(data,epochs=7,lrs=[0.5,0.3,0.2,0.15],frs=[0.02,0.01])
dbm.finetune(data,epochs=10,lrs=[0.5,0.5,0.4,0.3,0.2,0.1])

Pretraining layer 0:
    Epoch 0: cost = 0.809682
    Epoch 1: cost = 0.844806
    Epoch 2: cost = 0.870872
    Epoch 3: cost = 0.909047
    Epoch 4: cost = 0.911220
    Epoch 5: cost = 0.903291
    Epoch 6: cost = 0.915502
Pretraining layer 1:
    Epoch 0: cost = 1.257095
    Epoch 1: cost = 1.228000
    Epoch 2: cost = 1.285014
    Epoch 3: cost = 1.299890
    Epoch 4: cost = 1.260071
    Epoch 5: cost = 1.371015
    Epoch 6: cost = 1.353230
Pretraining layer 2:
    Epoch 0: cost = 1.811149
    Epoch 1: cost = 1.980749
    Epoch 2: cost = 2.082329
    Epoch 3: cost = 2.106684
    Epoch 4: cost = 2.034293
    Epoch 5: cost = 2.093188
    Epoch 6: cost = 2.085251
    Epoch 0: cost = 0.419986
    Epoch 1: cost = 0.466765
    Epoch 2: cost = 0.414422
    Epoch 3: cost = 0.423329
    Epoch 4: cost = 0.401117
    Epoch 5: cost = 0.384424
    Epoch 6: cost = 0.370906
    Epoch 7: cost = 0.362722
    Epoch 8: cost = 0.357901
    Epoch 9: cost = 0.354881


In [94]:
for rbm in dbm.rbms:
    print(rbm.W.get_value())

[[ 0.343  0.003  0.000  0.340]
 [ 0.234  0.002  0.000  0.394]
 [ 0.092  0.001  0.000  0.308]
 [ 0.276  0.002  0.000  0.645]
 [ 0.000  0.000  0.000  0.353]
 [ 0.000  0.000  0.000  0.589]
 [ 0.000  0.000  0.000  0.408]
 [ 0.000  0.000  0.000  0.592]]
[[ 0.000  0.000]
 [ 0.000  0.000]
 [ 0.035  0.000]
 [ 0.004  0.000]]
[[ 0.000]
 [ 0.000]]


In [76]:
f = theano.function([dbm.input],dbm.rbms[1].output)
f(data.dataset[0:4])

array([[ 1.000,  1.000],
       [ 1.000,  1.000],
       [ 1.000,  1.000],
       [ 1.000,  1.000]])

In [99]:
data.dataset[0:5]

array([[ 1.000,  1.000, -1.000,  1.000,  1.000, -1.000, -1.000,  1.000],
       [ 1.000,  1.000,  1.000,  1.000,  1.000,  1.000,  1.000,  1.000],
       [-1.000, -1.000, -1.000, -1.000,  1.000, -1.000,  1.000, -1.000],
       [ 1.000, -1.000,  1.000, -1.000, -1.000, -1.000, -1.000, -1.000],
       [ 1.000,  1.000,  1.000,  1.000,  1.000,  1.000,  1.000,  1.000]])

In [91]:
ent.rand_configs(200,subset=train,beta=2.)

array([[ 1,  1,  1, ...,  1,  1,  1],
       [ 1, -1,  1, ...,  1, -1, -1],
       [ 1, -1, -1, ...,  1, -1, -1],
       ..., 
       [ 1, -1, -1, ..., -1,  1, -1],
       [ 1, -1, -1, ..., -1, -1,  1],
       [-1, -1,  1, ..., -1, -1, -1]])